In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from resizeimage import resizeimage
from sklearn.svm import SVC
import pickle
from scipy import misc
from PIL import Image
import PIL
import imageio
from skimage import transform

In [2]:
data = pd.read_csv('train.csv')
features = data.iloc[:7500,1:]
labels = data.iloc[:7500,0]

In [3]:
for i in range(len(features)):
    digit = features.iloc[i]
    digit[digit<np.mean(digit)]=0
    digit[digit>=np.mean(digit)]=255
for i in range(len(features)):
    digit = features.iloc[i]
    digit = digit.reshape((28,28))
    del_arr = []
    for j in range(28):
        if 255 not in digit[j]:
            del_arr.append(j)
    digit = np.delete(digit,del_arr,0)
    #removing columns
    del_arr = []
    digit = digit.T
    for k in range(28):
        if 255 not in digit[k]:
            del_arr.append(k)
    digit = np.delete(digit,del_arr,0)
    digit = digit.T
    misc.imsave('modified/'+str(i)+'.jpg',digit)

/home/dante/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/home/dante/.local/lib/python3.5/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [4]:
dataset_modified = pd.DataFrame(index=None,columns=None)
for i in range(len(features)):
    img = imageio.imread('modified/'+str(i)+'.jpg')
    img = transform.resize(img,(28,28))
    img = img.astype(features.dtypes)
    img = misc.bytescale(img)
    img_test = []
    for eachRow in img:
        for eachPixel in eachRow:
            img_test.append(eachPixel)
    dataset_modified = dataset_modified.append(pd.Series(img_test),ignore_index=True)
dataset_modified.insert(0,column='labels',value=labels)

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/dante/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  


In [ ]:
new_features = dataset_modified.iloc[:7500,1:]
new_labels = dataset_modified.iloc[:7500,0]

In [ ]:
for i in range(len(new_features)):
    digit = new_features.iloc[i]
    digit[digit<np.mean(digit)]=0
    digit[digit>=np.mean(digit)]=255

In [ ]:
svm_clf1 = SVC(kernel='poly',degree=5,random_state=0)
svm_clf1.fit(new_features,new_labels)

In [ ]:
pickle.dump(svm_clf1,open('final_model.sav','wb'))
features.dtypes.to_csv('features_dtypes.csv')

In [ ]:
img = imageio.imread('out_snapsave.jpg')
img = transform.resize(img, (28,28))
img = img.astype(features.dtypes)
img = misc.bytescale(img)
x_test = []
for eachRow in img:
    for eachPixel in eachRow:
        x_test.append(eachPixel)
x_test = np.array(x_test)
#Binarization
x_test[x_test<225]=0
x_test[x_test>=225]=255
#Removing rows
x_test = x_test.reshape((28,28))
array = []
del_arr=[]
for i in range(len(x_test)):
    if 255 not in x_test[i]:
        del_arr.append(i)
x_test = np.delete(x_test,del_arr,0)
#Removing Columns
x_test=x_test.T
del_arr=[]
for i in range(len(x_test)):
    if 255 not in x_test[i]:
        del_arr.append(i)
x_test = np.delete(x_test,del_arr,0)
x_test = x_test.T
misc.imsave('error.jpg',x_test)
img2 = Image.open('error.jpg')
wpercent = (28/float(img2.size[0]))
hsize = int((float(img2.size[1])*float(wpercent)))
img3 = img2.resize((28,28),PIL.Image.ANTIALIAS)
img3.save('error3.jpg',img2.format)

In [ ]:
img = imageio.imread('error3.jpg')
img = transform.resize(img,(28,28))
img = img.astype(features.dtypes)
img = misc.bytescale(img)
img_test = []
for eachRow in img:
    for eachPixel in eachRow:
        img_test.append(eachPixel)
img_test = np.array(img_test)
img_test = img_test.reshape([1,784])
img_test[img_test<np.mean(img_test)]=0
img_test[img_test>=np.mean(img_test)]=255

In [ ]:
svm_clf1.predict(img_test)